In [9]:
import json
import requests

def fetch_miners_data(block_heights):
    """
    Fetch miners data from the API.
    
    Args:
        block_heights (list): List of block heights to query
    
    Returns:
        dict: JSON response from the API
    """
    base_url = "http://5.78.102.130:8000/sigscore/miners/average-participation"
    
    # Convert block heights list to comma-separated string
    blocks_param = ",".join(map(str, block_heights))
    
    # Make the API request
    response = requests.get(f"{base_url}?blocks={blocks_param}")
    response.raise_for_status()  # Raise exception for bad status codes
    
    return response.json()

def generate_distribution(miners_data, total_amount, token_name):
    """
    Generate a distribution JSON based on participation percentages.
    
    Args:
        miners_data (dict): JSON data containing miner information
        total_amount (float): Total amount to distribute
        token_name (str): Name of the token being distributed
    
    Returns:
        dict: Distribution JSON with calculated amounts
    """
    # Create recipients list with calculated amounts
    recipients = []
    
    for miner in miners_data['miners']:
        # Calculate amount based on participation percentage
        percentage = miner['avg_participation_percentage'] / 100.0
        amount = round(total_amount * percentage, 8)  # Round to 8 decimal places
        
        # Only include miners with non-zero amounts
        if amount > 0:
            recipients.append({
                "address": miner['miner_address'],
                "amount": amount
            })
    
    # Create the final distribution structure
    distribution = {
        "distributions": [
            {
                "token_name": token_name,
                "recipients": recipients
            }
        ]
    }
    
    return distribution

def save_distribution_json(distribution, output_file):
    """
    Save distribution data to a JSON file.
    
    Args:
        distribution (dict): Distribution data to save
        output_file (str): Path to the output JSON file
    """
    with open(output_file, 'w') as f:
        json.dump(distribution, f, indent=2)



In [12]:
def main():
    # Example block heights from your query
    block_heights = [1449288, 1449870, 1450108, 1450190, 1450325, 1450626,
                     1451319, 1451897, 1453049, 1454045, 1454583, 1454775]
    
    try:
        # Fetch data from API
        miners_data = fetch_miners_data(block_heights)
        
        # Generate distribution with example values
        total_amount = 3.9  # Example amount
        token_name = "ergo"  # Example token name
        
        result = generate_distribution(miners_data, total_amount, token_name)
        
        # Pretty print the result to console
        print(json.dumps(result, indent=2))
        
        # Save to JSON file
        output_file = "distribution_output.json"
        save_distribution_json(result, output_file)
        print(f"\nDistribution data saved to {output_file}")
        
    except requests.RequestException as e:
        print(f"Error fetching data from API: {e}")
    except Exception as e:
        print(f"Error processing data: {e}")

if __name__ == "__main__":
    main()

{
  "distributions": [
    {
      "token_name": "ergo",
      "recipients": [
        {
          "address": "9f6PPoCYrvHQbS6U8s7y6wE1S6pFET712ajBCzH5Ep7mYTGBudy",
          "amount": 0.9504807
        },
        {
          "address": "9eoPRhRmJQ71TPSkk2eUmMXYpxUVvZvFQo5uB9qjPBM4e3hC82J",
          "amount": 0.4758351
        },
        {
          "address": "9f3FRr4XudxVs1V35At1X5yj7LmQmnWqG46LqFKVNRf2TuYsAik",
          "amount": 0.2382744
        },
        {
          "address": "9hcqQ7GR8J9Uwd4gfiZGF25TXp1UqKdwoySNX9QEjuFcGgrM3mL",
          "amount": 0.1741623
        },
        {
          "address": "9fUBHUXfduFkCvHcx1eMV799a8uUp3wxYh4WkNNLwhLtURmPMWh",
          "amount": 0.1588977
        },
        {
          "address": "9i4DWcdw6c4PMdSHvBRJs2aPQ5rqzd4R4Vf1JPxeVi7wsZVd4c1",
          "amount": 0.1533246
        },
        {
          "address": "9fRQ8iCiCZekAewtjHmu7AmucWjvBTNUMeBFtnNkpfyWsNLUuxo",
          "amount": 0.1086618
        },
        {
          "address": "

In [24]:
import os
import requests
from typing import List
import json

class BlockHeightCollector:
    def __init__(self, wallet_address: str = "9fE5o7913CKKe6wvNgM11vULjTuKiopPcvCaj7t2zcJWXM2gcLu"):
        self.api_base = "https://api.ergoplatform.com/api/v1"
        self.wallet_address = os.getenv('WALLET_ADDRESS', wallet_address)

    def get_latest_outgoing_block(self) -> int:
        try:
            response = requests.get(
                f"{self.api_base}/addresses/{self.wallet_address}/transactions",
                params={"offset": 0, "limit": 50}
            )
            response.raise_for_status()
            
            for tx in response.json()["items"]:
                if any(input_data.get("address") == self.wallet_address 
                       for input_data in tx.get("inputs", [])):
                    return tx["inclusionHeight"]
            return 0
        except Exception as e:
            print(f"Error finding latest outgoing transaction: {e}")
            return 0

    def get_blocks_since_last_outgoing(self) -> List[int]:
        try:
            min_block = self.get_latest_outgoing_block()
            min_block = 1449260
            
            if not min_block:
                return []

            all_blocks = set()
            current_offset = 0
            limit = 100
            
            while True:
                response = requests.get(
                    f"{self.api_base}/addresses/{self.wallet_address}/transactions",
                    params={
                        "offset": current_offset,
                        "limit": limit,
                        "concise": True
                    }
                )
                response.raise_for_status()
                data = response.json()
                
                if not data["items"]:
                    break
                    
                for tx in data["items"]:
                    if tx.get("inclusionHeight", 0) > min_block:
                        all_blocks.add(tx["inclusionHeight"])
                        
                if len(data["items"]) < limit:
                    break
                    
                current_offset += limit
                
            return sorted(list(all_blocks))
        except Exception as e:
            print(f"Error fetching blocks: {e}")
            return []

def main():
    collector = BlockHeightCollector()
    block_heights = collector.get_blocks_since_last_outgoing()
    
    if not block_heights:
        print("No blocks found since last outgoing transaction")
        return
    
    try:
        miners_data = fetch_miners_data(block_heights)
        distribution = generate_distribution(miners_data, 3.9, "ergo")
        output_file = f"distribution_{len(block_heights)}_blocks.json"
        save_distribution_json(distribution, output_file)
        print(f"Distribution saved to {output_file}")
        
    except Exception as e:
        print(f"Error processing distribution: {e}")

if __name__ == "__main__":
    main()

Distribution saved to distribution_14_blocks.json
